In [1]:
import numpy as np
from scipy.integrate import quad, dblquad
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [2]:
thetamin = 17.8*np.pi/180
thetamax = 33.4*np.pi/180
t = 4*10**-6 #Cell Thickness

In [3]:
def compute_psi(E,alpha,p0):
    
    p0k = p0/1.38e-23
    
    #Scaling Parameters to try and help the best fit.
    p0k = p0k/(10**30)
    alpha = alpha/(10**10)
    
    def Boltz(theta,phi):
        return np.exp((1/T)*p0k*E*np.sin(theta)*np.cos(phi)*(1+alpha*E*np.cos(phi)))*np.sin(theta)

    def psi_fun(theta,phi):
        numerator = np.sin(2*theta)*np.cos(phi)
        denominator = np.cos(theta)**2 - np.sin(theta)**2 * np.cos(phi)**2
        boltz = np.exp((1/T)*p0k*E*np.sin(theta)*np.cos(phi)*(1+alpha*E*np.cos(phi)))
        return numerator*boltz*np.sin(theta)/denominator

    tan2psi= dblquad(psi_fun,0, 2*np.pi,lambda theta: thetamin, lambda theta: thetamax)[0]/10e100
    Z= dblquad(Boltz,0, 2*np.pi,lambda theta: thetamin, lambda theta: thetamax)[0]/10e100
    psi = np.arctan(tan2psi/Z)/2*180/np.pi
    
    return psi

In [4]:
def compute_error(xo):
    alpha = xo[0]
    p0 = xo[1]
    computed_psi = np.array([compute_psi(entry,alpha,p0) for entry in fields])
    Err = computed_psi - measured_psi
    ErrSqr = np.array([i**2 for i in Err])    
    return np.sum(ErrSqr)

In [21]:
temps = np.array([308])

Eum = np.array([2,3,6,7,9,11,12.5,14,16,18,20,22,23,25])
fields = np.array([i/(4*10**-6) for i in Eum])

psi1 = np.array([6,16,22,26,27.5,28,29,29.8,30,30.5,30.75,31,31.25,31.5])

# psi1 = np.array([[4.9056387, 16.85256143, 26.0563618 , 29.6809293, 31.14129687, 31.85484219, 32.26360885, 32.52501654, 32.70447992, 32.83371617]])
# psi2 = np.array([[4.36543608, 14.96461593, 24.4347061 ,28.81634912, 30.64663329, 31.53674634, 32.03723313, 32.35241811, 32.5670272, 32.72108479]])
# measured_psis = np.concatenate((psi1,psi2),axis=0)

measured_psis = psi1

results = np.array([])
xres=np.array([])

guess = (3000, 2000) #[alpha*10^10,p0*10^30]

Loop through an array of temperatures and compute the parameters at each temperature

In [22]:
for i in range(len(temps)):
    measured_psi=measured_psis[i]
    T = temps[i]
    bnds = ((2000, 5000), (1000, 4000))
    res = minimize(compute_error, guess,method = 'SLSQP', bounds=bnds)
    xres = np.array(dict(res.items())['x'])
    results = np.concatenate([results,xres], axis=0)
    guess = xres
results
    

array([ 2000.,  1000.])

In [23]:
results[0]*10**-4

0.20000000000000545

In [24]:
results[1]/3.33

300.30030030029423

In [9]:
# minimize(compute_error, guess, method='SLSQP', bounds=bnds)

TO DO:

LSA_parameters output is sloppy.

LSA_parameters needs to be able to run through a range of temperatures and for each temperature run through all of the fields. And return an array of all the computed parameters

Use parameters and plots V2 to make my own curve fit.

Look through the minimization methods and find one that works the best for this situation